In [2]:
import sys
import pandas as pd

middle of March - End of May 11 04 27 20 13 06 30 23

In [89]:
dates1 = [200530, 200523, 200516, 200509, 200502, 200425, 200418, 200411, 200404, 200328, 200321]
dates2 = [190525, 190518, 190511, 190504, 190427, 190420, 190413, 190406, 190330, 190323]
def make_df(dates):
    url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt'
    dfs = []
    for date in dates:
        link = url.format(date)
        dfx = pd.read_csv(link)
        dfs.append(dfx)
    df = pd.concat(dfs, ignore_index=True)
    for column in df.columns:
        df.rename(columns={column: column.lower().strip()}, inplace=True)
    return df

df_post = make_df(dates1)
df_pre = make_df(dates2)

In [34]:
df.to_csv('turnstiles.csv')

In [90]:
def add_datetime(df):
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%m/%d/%Y %H:%M:%S')
add_datetime(df_pre)
add_datetime(df_post)

In [94]:
df_post.head()

,scp,station,linename,date,time,entries,datetime
13458,05-06-01,FULTON ST,ACJZ2345,05/29/2020,23:00:00,886746,2020-05-29 23:00:00
13457,05-06-01,FULTON ST,ACJZ2345,05/29/2020,19:00:00,886746,2020-05-29 19:00:00
13456,05-06-01,FULTON ST,ACJZ2345,05/29/2020,15:00:00,886746,2020-05-29 15:00:00
13455,05-06-01,FULTON ST,ACJZ2345,05/29/2020,11:00:00,886746,2020-05-29 11:00:00
13454,05-06-01,FULTON ST,ACJZ2345,05/29/2020,07:00:00,886746,2020-05-29 07:00:00


In [95]:
df_pre.head()

,scp,station,linename,date,time,entries,datetime
12831,05-06-01,FULTON ST,ACJZ2345,05/24/2019,23:00:00,657200,2019-05-24 23:00:00
12830,05-06-01,FULTON ST,ACJZ2345,05/24/2019,19:00:00,657107,2019-05-24 19:00:00
12829,05-06-01,FULTON ST,ACJZ2345,05/24/2019,15:00:00,656624,2019-05-24 15:00:00
12828,05-06-01,FULTON ST,ACJZ2345,05/24/2019,11:00:00,656369,2019-05-24 11:00:00
12827,05-06-01,FULTON ST,ACJZ2345,05/24/2019,07:00:00,656281,2019-05-24 07:00:00


In [3]:
df_stations = pd.read_csv('stations_zip.csv')

In [93]:
def clean_df(df):
    df.drop(columns=['c/a','unit','exits','desc','division'], inplace=True)
    df.sort_values(['scp','station','datetime'], inplace=True, ascending=False)
    df.drop_duplicates(subset=['scp','station','datetime'], inplace=True)
clean_df(df_pre)
clean_df(df_post)

In [96]:
def make_daily_counts(df):
    df_daily_counts = df.groupby(['scp','station','date']).entries.first().reset_index()
    return df_daily_counts
pre_dailies = make_daily_counts(df_pre)
post_dailies = make_daily_counts(df_post)

In [101]:
post_dailies.head()

,scp,station,date,entries,prev_date,prev_entries
1,00-00-00,1 AV,05/12/2020,15455022,05/11/2020,15455022.0
2,00-00-00,1 AV,05/13/2020,15455022,05/12/2020,15455022.0
3,00-00-00,1 AV,05/14/2020,15455022,05/13/2020,15455022.0
4,00-00-00,1 AV,05/15/2020,15455022,05/14/2020,15455022.0
5,00-00-00,1 AV,05/16/2020,15455022,05/15/2020,15455022.0


In [102]:
pre_dailies.head()

,scp,station,date,entries,prev_date,prev_entries
1,00-00-00,1 AV,03/17/2019,14511518,03/16/2019,14511518.0
2,00-00-00,1 AV,03/18/2019,14513422,03/17/2019,14511518.0
3,00-00-00,1 AV,03/19/2019,14516664,03/18/2019,14513422.0
4,00-00-00,1 AV,03/20/2019,14520483,03/19/2019,14516664.0
5,00-00-00,1 AV,03/21/2019,14524635,03/20/2019,14520483.0


In [99]:
def add_prev(df):
    df[['prev_date', 'prev_entries']] = df.groupby(['scp','station'])[['date','entries']].apply(lambda line: line.shift(1))

add_prev(pre_dailies)
add_prev(post_dailies)

In [100]:
def rmna(df):
    df.dropna(subset=['prev_date'], axis=0, inplace=True)
rmna(pre_dailies)
rmna(post_dailies)

In [73]:
import numpy as np

In [106]:
def get_dailies(row):
    delta = row.entries - row.prev_entries
    if delta < 0:
        delta = -delta
    if delta > 100000:
        delta = min(row.entries, row.prev_entries)
    if delta > 100000:
        return np.nan
    return delta

pre_dailies['day_count'] = pre_dailies.apply(get_dailies, axis=1)
post_dailies['day_count'] = post_dailies.apply(get_dailies, axis=1)

In [107]:
post_dailies.head(20)

,scp,station,date,entries,prev_date,prev_entries,day_count
1,00-00-00,1 AV,05/12/2020,15455022,05/11/2020,15455022.0,0.0
2,00-00-00,1 AV,05/13/2020,15455022,05/12/2020,15455022.0,0.0
3,00-00-00,1 AV,05/14/2020,15455022,05/13/2020,15455022.0,0.0
4,00-00-00,1 AV,05/15/2020,15455022,05/14/2020,15455022.0,0.0
5,00-00-00,1 AV,05/16/2020,15455022,05/15/2020,15455022.0,0.0
6,00-00-00,1 AV,05/17/2020,15455022,05/16/2020,15455022.0,0.0
7,00-00-00,1 AV,05/18/2020,15455022,05/17/2020,15455022.0,0.0
8,00-00-00,1 AV,05/19/2020,15455022,05/18/2020,15455022.0,0.0
9,00-00-00,1 AV,05/20/2020,15455022,05/19/2020,15455022.0,0.0
10,00-00-00,1 AV,05/21/2020,15455023,05/20/2020,15455022.0,1.0


In [105]:
pre_dailies.head(21)

,scp,station,date,entries,prev_date,prev_entries,day_count
1,00-00-00,1 AV,03/17/2019,14511518,03/16/2019,14511518.0,0.0
2,00-00-00,1 AV,03/18/2019,14513422,03/17/2019,14511518.0,1904.0
3,00-00-00,1 AV,03/19/2019,14516664,03/18/2019,14513422.0,3242.0
4,00-00-00,1 AV,03/20/2019,14520483,03/19/2019,14516664.0,3819.0
5,00-00-00,1 AV,03/21/2019,14524635,03/20/2019,14520483.0,4152.0
6,00-00-00,1 AV,03/22/2019,14528718,03/21/2019,14524635.0,4083.0
7,00-00-00,1 AV,03/23/2019,14531081,03/22/2019,14528718.0,2363.0
8,00-00-00,1 AV,03/24/2019,14532765,03/23/2019,14531081.0,1684.0
9,00-00-00,1 AV,03/25/2019,14536261,03/24/2019,14532765.0,3496.0
10,00-00-00,1 AV,03/26/2019,14540002,03/25/2019,14536261.0,3741.0


In [108]:
pre_dailies.to_csv('2019_mta_counts')
post_dailies.to_csv('2020_mta_counts')